In [11]:
import financialStatement as fs
import pandas as pd

pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.expand_frame_repr', False)  # 긴 데이터 프레임 줄바꿈 없이 출력

stockInfo = fs.FinancialStatement()
stockInfo.setCompany('AAPL')
display(stockInfo.stock.info)

ReadTimeout: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Read timed out. (read timeout=30)

In [8]:
# 조건 1: current_ratio에서 2022년부터 2024년까지 값이 증가한 기업을 찾는 함수
def check_current_ratio_increase(row):
    # 2024년 값이 0이거나 None이라면 2023년을 최신값으로 간주
    if pd.isna(row[2024]) or row[2024] == 0:
        years = [2022, 2023]  # 2022년부터 2023년까지 값을 비교
        values = row[years].values
        return values[0] < values[1]  # 2022년 값 < 2023년 값
    else:
        years = [2022, 2023, 2024]  # 2022년부터 2024년까지 값을 비교
        values = row[years].values
        return all(values[i] < values[i + 1] for i in range(len(values) - 1))  # 값이 증가한 경우

# 조건 2: 최신 데이터(2024년 또는 2023년)가 2 이상인지 확인하는 함수
def check_current_ratio_latest(row):
    latest_year = 2024 if pd.notnull(row[2024]) and row[2024] != 0 else 2023
    latest_value = row[latest_year]
    return latest_value >= 2

# 조건 3: debt_to_equity_ratio에서 2022년부터 2024년까지 값이 감소한 기업을 찾는 함수
def check_debt_to_equity_ratio_decrease(row):
    # 2024년 값이 0이거나 None이라면 2023년을 최신값으로 간주
    if pd.isna(row[2024]) or row[2024] == 0:
        years = [2022, 2023]  # 2022년부터 2023년까지 값을 비교
        values = row[years].values
        return values[0] > values[1]  # 2022년 값 > 2023년 값
    else:
        years = [2022, 2023, 2024]  # 2022년부터 2024년까지 값을 비교
        values = row[years].values
        return all(values[i] > values[i + 1] for i in range(len(values) - 1))  # 값이 감소한 경우

# 조건 4: 부채비율의 최신 값이 1 이하인지 확인하는 함수
def check_debt_to_equity_latest(row):
    latest_year = 2024 if pd.notnull(row[2024]) and row[2024] != 0 else 2023
    latest_value = row[latest_year]
    return latest_value <= 1

# 티커 리스트 추출
valid_tickers = []

for idx, row in current_ratio.iterrows():
    # 조건 1: current_ratio 증가
    if check_current_ratio_increase(row) and check_current_ratio_latest(row):
        # 조건 3: debt_to_equity_ratio 감소
        debt_row = debt_to_equity_ratio.loc[debt_to_equity_ratio['Ticker'] == row['Ticker']].squeeze()
        if check_debt_to_equity_ratio_decrease(debt_row) and check_debt_to_equity_latest(debt_row):
            valid_tickers.append(row['Ticker'])

print("조건을 만족하는 티커들:", valid_tickers)


조건을 만족하는 티커들: ['NDRA', 'GCTK', 'HOLO', 'RIME', 'CTNT', 'SBFM', 'ELAB', 'NKLA', 'CLEU', 'VCIG', 'FTEL', 'SWIN', 'AEHL', 'LICN', 'IFBD', 'BHAT', 'RSLS', 'LGCB', 'JDZG', 'VSSYW', 'REVB', 'TCRT', 'FEBO', 'CABA', 'GXAI', 'UOKA', 'OTRK', 'UBXG', 'LGCL', 'ELEV', 'SPRC', 'ATPC', 'GLMD', 'STTK', 'XRTX', 'CLYM', 'FDMT', 'INO', 'CNEY', 'ASST', 'INAB', 'CRBU', 'FATE', 'AQMS', 'JL', 'LUCY', 'ALAR', 'SYRA', 'SURG', 'PLRX', 'RPTX', 'SCWO', 'WAI', 'BMEA', 'DWTX', 'SLRX', 'JSPR', 'GLTO', 'TNGX', 'NXTT', 'TIVC', 'MRNA', 'EYPT', 'VS', 'SUGP', 'BBLGW', 'CHEK', 'SEDG', 'OBLG', 'BAOS', 'MTVA', 'BBLG', 'TTNP', 'THAR', 'ANY', 'MGRX', 'LIPO', 'MRKR', 'WETH', 'YMAB', 'HOTH', 'FULC', 'SABS', 'GFAI', 'BGLC', 'SHLT', 'CYCN', 'UCAR', 'NAOV', 'TZUP', 'IMNM', 'SNES', 'MNYWW', 'BEEM', 'KOSS', 'SMSI', 'ANNX', 'OP', 'VKTX', 'SCYX', 'DOYU', 'MNY', 'RLAY', 'IPM', 'HDSN', 'EBON', 'BDMD', 'ARCT', 'VYGR', 'ACLS', 'JNVR', 'TLPH', 'JFU']


In [13]:
import pandas as pd

# 최신 데이터 찾는 함수
def get_latest_value(row):
    for year in ["2024", "2023", "2022", "2021", "2020"]:
        if year in row and pd.notna(row[year]) and row[year] != 0:
            return row[year]
    return None  # 모든 값이 NaN 또는 0이면 None 반환

# 특정 재무 지표를 처리하는 함수
def get_latest_financial_data(df, value_column_name):
    """
    df: 재무 데이터를 포함한 DataFrame
    value_column_name: 최종 컬럼 이름 (예: "Current Ratio" 또는 "Debt to Equity Ratio")
    """
    # 컬럼명을 문자열로 변환
    df.columns = df.columns.astype(str)

    df[value_column_name] = df.apply(get_latest_value, axis=1)
    return df[["Ticker", value_column_name, "Sector", "Industry"]] if "Sector" in df.columns else df[["Ticker", value_column_name]]

# 엑셀 파일 로드
current_ratio = pd.read_excel("nasdaq_current_ratio_1000.xlsx")
debt_to_equity_ratio = pd.read_excel("nasdaq_Debt-to-Equity Ratio_1000.xlsx")

# 각 데이터프레임 처리
current_ratio = get_latest_financial_data(current_ratio, "Current Ratio")
debt_to_equity_ratio = get_latest_financial_data(debt_to_equity_ratio, "Debt to Equity Ratio")

# 두 데이터프레임을 'Ticker' 기준으로 병합
final_df = pd.merge(current_ratio, debt_to_equity_ratio, on="Ticker")

# Sector와 Industry 컬럼 정리 (_x만 유지)
final_df = final_df.rename(columns={"Sector_x": "Sector", "Industry_x": "Industry"})
final_df = final_df[["Ticker", "Current Ratio", "Debt to Equity Ratio", "Sector", "Industry"]]

# 결과 확인
print(final_df.head())


  Ticker  Current Ratio  Debt to Equity Ratio             Sector  \
0   ADTX       0.267014              1.776961         Healthcare   
1   TRNR       0.248723                   NaN  Consumer Cyclical   
2   CRKN       0.495233              1.036990    Basic Materials   
3   CETX       1.435838              8.311887         Technology   
4   MULN       0.344892              0.712591  Consumer Cyclical   

                    Industry  
0              Biotechnology  
1          Personal Services  
2        Specialty Chemicals  
3  Software - Infrastructure  
4         Auto Manufacturers  


In [14]:

valid_tickers = ['NDRA', 'GCTK', 'HOLO', 'RIME', 'CTNT', 'SBFM', 'ELAB', 'NKLA', 'CLEU', 'VCIG', 'FTEL', 'SWIN', 'AEHL', 'LICN', 'IFBD', 'BHAT', 'RSLS', 'LGCB', 'JDZG', 'VSSYW', 'REVB', 'TCRT', 'FEBO', 'CABA', 'GXAI', 'UOKA', 'OTRK', 'UBXG', 'LGCL', 'ELEV', 'SPRC', 'ATPC', 'GLMD', 'STTK', 'XRTX', 'CLYM', 'FDMT', 'INO', 'CNEY', 'ASST', 'INAB', 'CRBU', 'FATE', 'AQMS', 'JL', 'LUCY', 'ALAR', 'SYRA', 'SURG', 'PLRX', 'RPTX', 'SCWO', 'WAI', 'BMEA', 'DWTX', 'SLRX', 'JSPR', 'GLTO', 'TNGX', 'NXTT', 'TIVC', 'MRNA', 'EYPT', 'VS', 'SUGP', 'BBLGW', 'CHEK', 'SEDG', 'OBLG', 'BAOS', 'MTVA', 'BBLG', 'TTNP', 'THAR', 'ANY', 'MGRX', 'LIPO', 'MRKR', 'WETH', 'YMAB', 'HOTH', 'FULC', 'SABS', 'GFAI', 'BGLC', 'SHLT', 'CYCN', 'UCAR', 'NAOV', 'TZUP', 'IMNM', 'SNES', 'MNYWW', 'BEEM', 'KOSS', 'SMSI', 'ANNX', 'OP', 'VKTX', 'SCYX', 'DOYU', 'MNY', 'RLAY', 'IPM', 'HDSN', 'EBON', 'BDMD', 'ARCT', 'VYGR', 'ACLS', 'JNVR', 'TLPH', 'JFU']
finish = []

stock = fs.FinancialStatement()

for ticker in valid_tickers:
    stock.setCompany(ticker)
    cash_flow = stock.getCashFlow()
    
    try:
        if cash_flow.loc['Net Common Stock Issuance'].iloc[0] <= 0:
            finish.append(ticker)

    except Exception as e:
        print(f"{ticker} 에러발생 : {e}")


filtered_df = final_df[final_df["Ticker"].isin(finish)]
filtered_df

HOLO 에러발생 : 'Net Common Stock Issuance'
LICN 에러발생 : 'Net Common Stock Issuance'
LGCL 에러발생 : 'Net Common Stock Issuance'
MNYWW 에러발생 : 'Net Common Stock Issuance'
KOSS 에러발생 : 'Net Common Stock Issuance'
MNY 에러발생 : 'Net Common Stock Issuance'
BDMD 에러발생 : 'Net Common Stock Issuance'


,Ticker,Current Ratio,Debt to Equity Ratio,Sector,Industry
272,XRTX,13.795384,0.177926,Healthcare,Biotechnology
351,CNEY,42.410197,0.014963,Basic Materials,Specialty Chemicals
357,ASST,19.297803,0.052541,Communication Services,Internet Content & Information
497,MRNA,3.424544,0.330013,Healthcare,Biotechnology
529,CHEK,18.688722,0.056149,Healthcare,Diagnostics & Research
534,SEDG,3.699274,0.902116,Technology,Solar
555,BAOS,6.110553,0.152682,Communication Services,Advertising Agencies
585,TTNP,5.552632,0.217404,Healthcare,Biotechnology
616,LIPO,8.899015,0.137616,Healthcare,Biotechnology
681,SABS,5.447622,0.464958,Healthcare,Biotechnology
